In [1]:
import pyforest
from rdkit import Chem
from rdkit.Chem import AllChem
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE 

In [3]:
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/'
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'

In [71]:
df = pd.read_csv(datapath+'final_pIC50s_for_ML_noOut_noDupes_pivot.csv')
print(df.shape)
df=df.dropna()
print(df.shape)


<IPython.core.display.Javascript object>

(400, 10)
(400, 10)


In [72]:
df['smiles'].unique().shape

(400,)

In [33]:
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/original_datasets/'
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition']
all_smiles_lists = []
for nek in [2, 3, 5, 9]: 
    nek_type_list = ['binding']
    if nek in [2, 9]: 
        nek_type_list =['binding', 'inhibition']
    for bi in nek_type_list: 
        nek_df = pd.read_csv(f'{og_datapath}NEK{nek}_1_uM_min_50_pct_{bi}.csv')
        all_smiles_lists.append(nek_df['base_rdkit_smiles'].values)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
all_smiles = []
for sublist in all_smiles_lists:
    all_smiles.extend(sublist)
len(all_smiles)

In [55]:
mol_in_og_data = df[df['smiles'].isin(all_smiles)]
new_df = df[~df['smiles'].isin(all_smiles)]
mol_in_og_data
new_df
print(f'new nanobret data size after filtering: {new_df.shape}')
for nek in [ x for x in new_df.columns if x.startswith('NEK')]: 
    new_df[f'{nek}_active']=new_df[nek].apply(lambda x: 1 if x>6 else 0)

new nanobret data size after filtering: (400, 10)


In [70]:
for nek in [ x for x in new_df.columns if x.endswith('active')]: 
    nek_str = nek.split('_')[0]

    cols = ['Compound', 'smiles', nek] 
    nek_df = new_df[cols]
    # display(nek_df)
    display(nek_df[nek].value_counts())
    nek_df.to_csv(f'{datapath}{nek_str}_uM_min_50_pct.csv', index=False)

NEK1_active
0    370
1     30
Name: count, dtype: int64

NEK11_active
0    398
1      2
Name: count, dtype: int64

NEK2_active
0    369
1     31
Name: count, dtype: int64

NEK3_active
0    372
1     28
Name: count, dtype: int64

NEK4_active
0    400
Name: count, dtype: int64

NEK5_active
0    320
1     80
Name: count, dtype: int64

NEK6_active
0    400
Name: count, dtype: int64

NEK9_active
0    366
1     34
Name: count, dtype: int64

# Next steps
1. figure out binding or inhibition data --> pIC50 of 6 is equivalent to 50% inhibition at 1uM
2. how should we convert the labels to active/inactive? (our previous datasets were perecent binding/inhibition)
4. run this new data on our production models 